# lmports and definitions

In [2]:
# variable       delta_coherence_calculation 
# Class          Captial
# function       butterBandpassFilter
# meegkiit install -- command window
# variable names -- solved - next meeting
# definition of function -- solved - next meeting
# Visualization interpretation -- future
# welch - methods extraction -- future 
# Introduction to asymetry, coherence , ... by Pegah

# parameter      delta_coherence_calculation
# variable       var_delta_coherence_calculation 
# Class          Captial
# function       butterBandpassFilter
# internal variables or parameter name with _ at the beginning eg: _counter
import mne
import pandas as pd
import numpy as np
import scipy.io
import scipy.signal as ss
import pyedflib
from scipy.signal import hilbert
from scipy.signal import butter, lfilter
from scipy.signal import welch
from scipy.integrate import simps
from meegkit.asr import ASR
from meegkit.utils.matrix import sliding_window
from mne.time_frequency import psd_array_multitaper
from neurodsp.utils.download import load_ndsp_data
#frequency rythms
#Delta:0.5-4
#Theta:4-8
#Alpha: 8-13
#Beta: 13-30
#Gamma:30-45
#eeg channels corresponding brain area
# left frontal (Fl)=fp1,f3,f7=0,2,10
# right frontal (Fr)=fp2,f4,f8=1,3,11
# left central (C)=t3, c3=12,4
# right central (C)=t4, c4=13,5
# left parietal-occipital (lPO)=t5,p3,o1=14,6,8
# right parietal-occipital (rPO)=t6,p4,o2=15,7,9
# midline (M)=fz,cz,pz=17,16,18
######################################################################################################
fs=1000
def butter_bandpass(lowcut, highcut, fs, order=4):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a
def butter_bandpass_filter(data, lowcut, highcut, fs, order=4):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

def apply_asr(raw, sfreq=fs, cutoff=2.5, blocksize=100, win_len=0.5,
        win_overlap=0.66, max_dropout_fraction=0.1,
        min_clean_fraction=0.25, name='asrfilter', method='euclid',
        estimator='scm'):
    h, w = raw.shape
    
    # t = int(raw.shape[1] / sfreq)
    asr = ASR(sfreq=sfreq, cutoff=cutoff, blocksize=blocksize, win_len=win_len,
        win_overlap=win_overlap, max_dropout_fraction=max_dropout_fraction,
        min_clean_fraction=min_clean_fraction, name=name, method=method,
        estimator=estimator)
        
        # method='euclid'
    # train_idx = np.arange(0 * sfreq, t * sfreq, dtype=int)
    _, sample_mask = asr.fit(raw)
    
    # Apply filter using sliding (non-overlapping) windows
    X = sliding_window(raw, window=int(sfreq), step=int(sfreq))
    Y = np.zeros_like(X)
    for i in range(X.shape[1]):
        Y[:, i, :] = asr.transform(X[:, i, :])
    
    raw = X.reshape(h, -1)  # reshape to (n_chans, n_times)
    clean = Y.reshape(h, -1)
    return raw, clean
def bandpower(data, sf, band, method='welch', window_sec=None, relative=False):
    band = np.asarray(band)
    low, high = band
    # Compute the modified periodogram (Welch)
    if method == 'welch':
        if window_sec is not None:
            nperseg = window_sec * sf
        else:
            nperseg = (2 / low) * sf
        freqs, psd = welch(data, sf, nperseg=nperseg)
    elif method == 'multitaper':
        psd, freqs = psd_array_multitaper(data, sf, adaptive=True,
                                          normalization='full', verbose=0)
    # Frequency resolution
    freq_res = freqs[1] - freqs[0]

    # Find index of band in frequency vector
    idx_band = np.logical_and(freqs >= low, freqs <= high)

    # Integral approximation of the spectrum using parabola (Simpson's rule)
    bp = simps(psd[idx_band], dx=freq_res)

    if relative:
        bp /= simps(psd, dx=freq_res)
    return bp
def calc_rms(x, scale):
    # making an array with data divided in windows
    shape = (x.shape[0]//scale, scale)
    X = np.lib.stride_tricks.as_strided(x,shape=shape)
    # vector of x-axis points to regression
    scale_ax = np.arange(scale)
    rms = np.zeros(X.shape[0])
    for e, xcut in enumerate(X):
        coeff = np.polyfit(scale_ax, xcut, 1)
        xfit = np.polyval(coeff, scale_ax)
        # detrending and computing RMS of each window
        rms[e] = np.sqrt(np.mean((xcut-xfit)**2))
    return rms

def dfa(x, scale_lim=[5,9], scale_dens=0.128, show=False):
    # cumulative sum of data with substracted offset
    y = np.cumsum(x - np.mean(x))
    scales = (2**np.arange(scale_lim[0], scale_lim[1], scale_dens)).astype(np.int)
    fluct = np.zeros(len(scales))
    # computing RMS for each window
    for e, sc in enumerate(scales):
        fluct[e] = np.sqrt(np.mean(calc_rms(y, sc)**2))
    # fitting a line to rms data
    coeff = np.polyfit(np.log2(scales), np.log2(fluct), 1)
    if show:
        fluctfit = 2**np.polyval(coeff,np.log2(scales))
    return scales, fluct, coeff[0]

KeyboardInterrupt: 

# load and preprocess

In [ ]:
raw= mne.io.read_raw_eeglab('D:\WM_openneuro_dataset\sub-064\eeg\sub-064_task-rest_eeg.set')
all_events, all_event_id = mne.events_from_annotations(raw)
my_events= mne.pick_events(all_events)
epochs = mne.Epochs(raw,my_events, tmin=0.0, tmax=240.0, baseline=(0, 0))
data=epochs.to_data_frame()
data.drop('time', axis=1, inplace=True)
data.drop('condition', axis=1, inplace=True)
data.drop('epoch', axis=1, inplace=True)
data.drop([0],axis=0,inplace=True)
edata=data.to_numpy(dtype ='float32')
clm,rw =data.shape
eeg=np.reshape(edata,(rw,-1))

filter_signal = butter_bandpass_filter(eeg, 1, 45, 1000)
signal, filter_asr_signal = apply_asr(filter_signal)
#np.save('s64_denoise_resting_state', filter_asr_signal)

# absolout and relative band power

In [ ]:
delta_p = delta_p_rel = theta_p=theta_p_rel=alpha_p=alpha_p_rel=beta_p=beta_p_rel=gamma_p= gamma_p_rel=np.zeros(21)
for ii in range (0,20):
    delta_p[ii] = bandpower(filter_asr_signal[ii,:], fs, [1, 4], 'multitaper')
    delta_p_rel[ii] = bandpower(filter_asr_signal[ii,:], fs, [1, 4], 'multitaper', relative=True)
    theta_p[ii]= bandpower(filter_asr_signal[ii,:], fs, [4, 8], 'multitaper')
    theta_p_rel[ii] = bandpower(filter_asr_signal[ii,:], fs, [4, 8], 'multitaper', relative=True)
    alpha_p[ii] = bandpower(filter_asr_signal[ii,:], fs, [8, 12], 'multitaper')
    alpha_p_rel[ii] = bandpower(filter_asr_signal[ii,:], fs, [8, 12], 'multitaper', relative=True)
    beta_p[ii] = bandpower(filter_asr_signal[ii,:], fs, [12, 25], 'multitaper')
    beta_p_rel[ii] = bandpower(filter_asr_signal[ii,:], fs, [12, 25], 'multitaper', relative=True)
    h_beta_p[ii] = bandpower(filter_asr_signal[ii,:], fs, [25, 30], 'multitaper')
    h_beta_p_rel[ii] = bandpower(filter_asr_signal[ii,:], fs, [25, 30], 'multitaper', relative=True)
    gamma_p[ii] = bandpower(filter_asr_signal[ii,:], fs, [30, 40], 'multitaper')
    gamma_p_rel[ii] = bandpower(filter_asr_signal[ii,:], fs, [30, 40], 'multitaper', relative=True)
    h_gamma_p[ii] = bandpower(filter_asr_signal[ii,:], fs, [40, 48], 'multitaper')
    h_gamma_p_rel[ii] = bandpower(filter_asr_signal[ii,:], fs, [40, 48], 'multitaper', relative=True)
    alpha_1_p[ii] = bandpower(filter_asr_signal[ii,:], fs, [8, 10], 'multitaper')
    alpha_1_p_rel[ii] = bandpower(filter_asr_signal[ii,:], fs, [8, 10], 'multitaper', relative=True)
    alpha_2_p[ii] = bandpower(filter_asr_signal[ii,:], fs, [10, 12], 'multitaper')
    alpha_2_p_rel[ii] = bandpower(filter_asr_signal[ii,:], fs, [10, 12], 'multitaper', relative=True)
    beta_1_p[ii] = bandpower(filter_asr_signal[ii,:], fs, [12, 15], 'multitaper')
    beta_1_p_rel[ii] = bandpower(filter_asr_signal[ii,:], fs, [12, 15], 'multitaper', relative=True)
    beta_2_p[ii] = bandpower(filter_asr_signal[ii,:], fs, [15, 18], 'multitaper')
    beta_2_p_rel[ii] = bandpower(filter_asr_signal[ii,:], fs, [15, 18], 'multitaper', relative=True)
    beta_3_p[ii] = bandpower(filter_asr_signal[ii,:], fs, [18, 25], 'multitaper')
    beta_3_p_rel[ii] = bandpower(filter_asr_signal[ii,:], fs, [18, 25], 'multitaper', relative=True)
    gama_1_p[ii] = bandpower(filter_asr_signal[ii,:], fs, [30, 35], 'multitaper')
    gama_1_p_rel[ii] = bandpower(filter_asr_signal[ii,:], fs, [30, 35], 'multitaper', relative=True)
    gama_2_p[ii] = bandpower(filter_asr_signal[ii,:], fs, [35, 40], 'multitaper')
    gama_2_p_rel[ii] = bandpower(filter_asr_signal[ii,:], fs, [35, 40], 'multitaper', relative=True)
#ratio-power
#db = bandpower(filter_asr_signal[1,:], fs, [0.5, 4], 'multitaper') / bandpower(filter_asr_signal[1,:], fs, [12, 30], 'multitaper')

# amplitude asymmetry

In [26]:
Amp_assym_delta=np.zeros((21,21))
Amp_assym_theta=np.zeros((21,21))
Amp_assym_alpha=np.zeros((21,21))
Amp_assym_beta=np.zeros((21,21))
Amp_assym_h_beta=np.zeros((21,21))
Amp_assym_beta1=np.zeros((21,21))
Amp_assym_beta2=np.zeros((21,21))
Amp_assym_beta3=np.zeros((21,21))

filter_signal_delta = butter_bandpass_filter(filter_asr_signal, 1, 4, 1000)
filter_signal_theta = butter_bandpass_filter(filter_asr_signal, 4, 8, 1000)
filter_signal_alpha = butter_bandpass_filter(filter_asr_signal, 8, 12, 1000)
filter_signal_beta = butter_bandpass_filter(filter_asr_signal, 12, 25, 1000)
filter_signal_h_beta = butter_bandpass_filter(filter_asr_signal, 25, 30, 1000)
filter_signal_beta1 = butter_bandpass_filter(filter_asr_signal, 12, 15, 1000)
filter_signal_beta2 = butter_bandpass_filter(filter_asr_signal, 15, 18, 1000)
filter_signal_beta3 = butter_bandpass_filter(filter_asr_signal, 18, 25, 1000)
for jj in range (0,20):
    for j in range (0,20):
        ch1_delta=filter_signal_delta[jj,:]
        ch2_delta=filter_signal_delta[j,:] 
        p_delta_1=np.log(np.mean(np.abs(ch1_delta**2)))
        p_delta_2=np.log(np.mean(np.abs(ch2_delta**2)))
        Amp_assym_delta[jj,j]=p_delta_1-p_delta_2
        
        ch1_theta=filter_signal_theta[jj,:]
        ch2_theta=filter_signal_theta[j,:] 
        p_theta_1=np.log(np.mean(np.abs(ch1_theta**2)))
        p_theta_2=np.log(np.mean(np.abs(ch2_theta**2)))
        Amp_assym_theta[jj,j]=p_theta_1-p_theta_2
        
        ch1_alpha=filter_signal_alpha[jj,:]
        ch2_alpha=filter_signal_alpha[j,:] 
        p_alpha_1=np.log(np.mean(np.abs(ch1_alpha**2)))
        p_alpha_2=np.log(np.mean(np.abs(ch2_alpha**2)))
        Amp_assym_alpha[jj,j]=p_alpha_1-p_alpha_2
        
        ch1_beta=filter_signal_beta[jj,:]
        ch2_beta=filter_signal_beta[j,:] 
        p_beta_1=np.log(np.mean(np.abs(ch1_beta**2)))
        p_beta_2=np.log(np.mean(np.abs(ch2_beta**2)))
        Amp_assym_beta[jj,j]=p_beta_1-p_beta_2
        
        ch1_h_beta=filter_signal_h_beta[jj,:]
        ch2_h_beta=filter_signal_h_beta[j,:] 
        p_h_beta_1=np.log(np.mean(np.abs(ch1_h_beta**2)))
        p_h_beta_2=np.log(np.mean(np.abs(ch2_h_beta**2)))
        Amp_assym_h_beta[jj,j]=p_h_beta_1-p_h_beta_2
        
        ch1_beta_1=filter_signal_beta1[jj,:]
        ch2_beta_1=filter_signal_beta1[j,:] 
        p_beta_1_1=np.log(np.mean(np.abs(ch1_beta_1**2)))
        p_beta_1_2=np.log(np.mean(np.abs(ch2_beta_1**2)))
        Amp_assym_beta1[jj,j]=p_beta_1_1-p_beta_1_2
        
        ch1_beta_2=filter_signal_beta2[jj,:]
        ch2_beta_2=filter_signal_beta2[j,:] 
        p_beta_2_1=np.log(np.mean(np.abs(ch1_beta_2**2)))
        p_beta_2_2=np.log(np.mean(np.abs(ch2_beta_2**2)))
        Amp_assym_beta2[jj,j]=p_beta_2_1-p_beta_2_2
        
        ch1_beta_3=filter_signal_beta3[jj,:]
        ch2_beta_3=filter_signal_beta3[j,:] 
        p_beta_3_1=np.log(np.mean(np.abs(ch1_beta_3**2)))
        p_beta_3_2=np.log(np.mean(np.abs(ch2_beta_3**2)))
        Amp_assym_beta3[jj,j]=p_beta_3_1-p_beta_3_2

# coherence

In [5]:
coherence1=np.zeros((21,21))
c_delta=np.zeros((4,21,21))
c_theta=np.zeros((4,21,21))
c_alpha=np.zeros((5,21,21))
c_beta=np.zeros((17,21,21))
c_gamma=np.zeros((15,21,21))
for jj in range (0,20):
    for j in range (0,20):
        ch_m=filter_asr_signal[jj,:]
        ch_n=filter_asr_signal[j,:]
        power_f=np.log(np.mean(np.abs(ch_m**2)))
        power_ff=np.log(np.mean(np.abs(ch_n**2)))
        fxx,cxx=ss.coherence(ch_m, ch_n, fs=1000, window='hann', nperseg=1000, noverlap=None, nfft=None, detrend='constant', axis=-1)
        c_delta[:,jj,j]=cxx[1:5]
        c_theta[:,jj,j]=cxx[5:9]
        c_alpha[:,jj,j]=cxx[9:13]
        c_beta[:,jj,j]=cxx[13:26]
        c_h_beta[:,jj,j]=cxx[26:31]
        c_beta1[:,jj,j]=cxx[13:16]
        c_beta2[:,jj,j]=cxx[16:19]
        c_beta3[:,jj,j]=cxx[19:26]

# phase lag

In [1]:
phase_diff=np.zeros((21,21))
for jj in range (0,20):
    for j in range (0,20):
        ch1_delta=filter_signal_delta[jj,:]
        ch2_delta=filter_signal_delta[j,:] 
        x1_delta= hilbert(ch1_delta)
        x2_delta = hilbert(ch2_delta)
        c_delta = np.inner( x1_delta, np.conj(x2_delta) )/np.sqrt( np.inner(x1_delta,np.conj(x1_delta)) * np.inner(x2_delta,np.conj(x2_delta)))
        phase_diff_delta[jj,j]= np.angle(c_delata)
       
        ch1_theta=filter_signal_theta[jj,:]
        ch2_theta=filter_signal_theta[j,:] 
        x1_theta= hilbert(ch1_theta)
        x2_theta = hilbert(ch2_theta)
        c_theta = np.inner( x1_theta, np.conj(x2_theta) )/np.sqrt( np.inner(x1_theta,np.conj(x1_theta)) * np.inner(x2_theta,np.conj(x2_theta)))
        phase_diff_theta[jj,j]= np.angle(c_theta)
       
        ch1_alpha=filter_signal_alpha[jj,:]
        ch2_alpha=filter_signal_alpha[j,:] 
        x1_alpha= hilbert(ch1_alpha)
        x2_alpha = hilbert(ch2_alpha)
        c_alpha = np.inner( x1_alpha, np.conj(x2_alpha) )/np.sqrt( np.inner(x1_alpha,np.conj(x1_alpha)) * np.inner(x2_alpha,np.conj(x2_alpha)))
        phase_diff_alpha[jj,j]= np.angle(c_alpha)
        
        ch1_beta=filter_signal_beta[jj,:]
        ch2_beta=filter_signal_beta[j,:] 
        x1_beta= hilbert(ch1_beta)
        x2_beta = hilbert(ch2_beta)
        c_beta= np.inner( x1_beta, np.conj(x2_beta) )/np.sqrt( np.inner(x1_beta,np.conj(x1_beta)) * np.inner(x2_beta,np.conj(x2_beta)))
        phase_diff_beta[jj,j]= np.angle(c_beta)
       
       
        ch1_h_beta=filter_signal_h_beta[jj,:]
        ch2_h_beta=filter_signal_h_beta[j,:] 
        x1_h_beta= hilbert(ch1_h_beta)
        x2_h_beta = hilbert(ch2_h_beta)
        c_h_beta = np.inner( x1_h_beta, np.conj(x2_h_beta) )/np.sqrt( np.inner(x1_h_beta,np.conj(x1_h_beta)) * np.inner(x2_h_beta,np.conj(x2_h_beta)))
        phase_diff_h_beta[jj,j]= np.angle(c_h_beta)
        
        ch1_beta1=filter_signal_beta1[jj,:]
        ch2_beta1=filter_signal_beta1[j,:] 
        x1_beta1= hilbert(ch1_beta1)
        x2_beta1 = hilbert(ch2_beta1)
        c_beta1= np.inner( x1_beta1, np.conj(x2_beta1) )/np.sqrt( np.inner(x1_beta1,np.conj(x1_beta1)) * np.inner(x2_beta1,np.conj(x2_beta1)))
        phase_diff_beta1[jj,j]= np.angle(c_beta1)
        
        ch1_beta2=filter_signal_beta2[jj,:]
        ch2_beta2=filter_signal_beta2[j,:] 
        x1_beta2= hilbert(ch1_beta2)
        x2_beta2 = hilbert(ch2_beta2)
        c_beta2 = np.inner( x1_beta2, np.conj(x2_beta2) )/np.sqrt( np.inner(x1_beta2,np.conj(x1_beta2)) * np.inner(x2_beta2,np.conj(x2_beta2)))
        phase_diff_beta2[jj,j]= np.angle(c_beta2)
       
        ch1_beta3=filter_signal_beta3[jj,:]
        ch2_beta3=filter_signal_beta3[j,:] 
        x1_beta3= hilbert(ch1_beta3)
        x2_beta3 = hilbert(ch2_beta3)
        c_beta3 = np.inner( x1_beta3, np.conj(x2_beta3) )/np.sqrt( np.inner(x1_beta3,np.conj(x1_beta3)) * np.inner(x2_beta3,np.conj(x2_beta3)))
        phase_diff_beta3[jj,j]= np.angle(c_beta3)
       

NameError: name 'np' is not defined

# DFA

In [ ]:
dfas=np.empty(21)
for i in range (21):
    if __name__=='__main__':
        x_delta = np.abs(ss.hilbert(filter_signal_delta[i]))
        scales, fluct, alpha_factor_delta = dfa(x_delta, show=1)
        dfas_delta[i]=alpha_factor_delta
        
         x_theta = np.abs(ss.hilbert(filter_signal_theta[i]))
        scales, fluct, alpha_factor_theta= dfa(x_theta, show=1)
        dfas_theta[i]=alpha_factor_theta
        
         x_alpha= np.abs(ss.hilbert(filter_signal_alpha[i]))
        scales, fluct, alpha_factor_alpha = dfa(x_alpha, show=1)
        dfas_alpha[i]=alpha_factor_alpha
        
         x_beta = np.abs(ss.hilbert(filter_signal_beta[i]))
        scales, fluct, alpha_factor_beta = dfa(x_beta, show=1)
        dfas_beta[i]=alpha_factor_beta
        
         x_h_beta = np.abs(ss.hilbert(filter_signal_h_beta[i]))
        scales, fluct, alpha_factor_h_beta = dfa(x_h_beta, show=1)
        dfas_h_beta[i]=alpha_factor_h_beta
        
         x_beta1 = np.abs(ss.hilbert(filter_signal_beta1[i]))
        scales, fluct, alpha_factor_beta1 = dfa(x_beta1, show=1)
        dfas_beta1[i]=alpha_factor_beta1
        
         x_beta2 = np.abs(ss.hilbert(filter_signal_beta2[i]))
        scales, fluct, alpha_factor_beta2 = dfa(x_beta2, show=1)
        dfas_beta2[i]=alpha_factor_bet2
        
         x_beta3 = np.abs(ss.hilbert(filter_signal_beta3[i]))
        scales, fluct, alpha_factor_beta3 = dfa(x_beta3, show=1)
        dfas_beta3[i]=alpha_factor_beta3
        